**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [17]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras import regularizers
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Dropout

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The function act takes the current state s, and returns the action a to play next. If the argument train is equal to False, the action is chosen according to the policy the agent already learnt. If the argument train is equal to True, the action is chosen according to the policy of the agent with probability $1 - \epsilon$, or randomly with probability $\epsilon$, thus allowing for exploration which is essential for the agent to learn an optimal policy. The essential parameter $\epsilon$ thus controls the exploration / exploitation tradeoff.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [22]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=200 # set small when debugging
epochs_test=50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> The array position is used to locate the rat on the game grid, while the array board is used to locate the cheese and poisonous cells. Used together, they allow, after taking an action i.e. moving the rat, to return the next state i.e. the 5x5 subgrid view of the game that the rat can see around its own absolute position.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.choice(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.5/13.0. Average score (-3.5)
Win/lose count 15.0/15.0. Average score (-1.75)
Win/lose count 9.0/14.0. Average score (-2.8333333333333335)
Win/lose count 10.0/8.0. Average score (-1.625)
Win/lose count 10.0/19.0. Average score (-3.1)
Win/lose count 11.5/18.0. Average score (-3.6666666666666665)
Win/lose count 12.0/22.0. Average score (-4.571428571428571)
Win/lose count 15.0/13.0. Average score (-3.75)
Win/lose count 10.0/15.0. Average score (-3.888888888888889)
Win/lose count 10.5/19.0. Average score (-4.35)
Win/lose count 9.0/13.0. Average score (-4.318181818181818)
Win/lose count 15.0/10.0. Average score (-3.5416666666666665)
Win/lose count 6.0/12.0. Average score (-3.730769230769231)
Win/lose count 8.5/18.0. Average score (-4.142857142857143)
Win/lose count 8.5/22.0. Average score (-4.766666666666667)
Win/lose count 7.0/16.0. Average score (-5.03125)
Win/lose count 8.5/15.0. Average score (-5.117647058823529)
Win/lose count 7.5/17.0. Average score (-5.361111111111111

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




> \begin{align*}
Q^\pi(s,a) =& \ E_{p^{\pi}}[\sum_{t = 0}^{+\infty}\gamma^{t}r(s_{t},a_{t}) \ | \ s_{0}=s,a_{0}=a] \\
=& \ E_{s_1 \sim p(.|s_0, a_0)}[E_{p^{\pi}}[\sum_{t = 0}^{+\infty}\gamma^{t}r(s_{t},a_{t}) \ | \ s_{0}=s,a_{0}=a, s_1]] \\
=& \ E_{s_1 \sim p(.|s_0, a_0)}[E_{p^{\pi}}[r(s_0, a_0) \ | \ s_{0}=s,a_{0}=a, s_1]]
+ \ E_{s_1 \sim p(.|s_0, a_0)}[E_{p^{\pi}}[\sum_{t = 1}^{+\infty}\gamma^{t}r(s_{t},a_{t}) \ | \ s_{0}=s,a_{0}=a, s_1]] \\
=& \ r(s, a)
+ \ E_{s_1 \sim p(.|s, a)}[E_{p^{\pi}}[\sum_{t = 1}^{+\infty}\gamma^{t}r(s_{t},a_{t}) \ | \ s_0 = s, a_0 = a, s_1]] \\
=& \ r(s, a)
+ \ E_{s_1 \sim p(.|s, a)}[E_{p^{\pi}}[\sum_{t = 1}^{+\infty}\gamma^{t}r(s_{t},a_{t}) \ | s_1]] \ \text{(Markov property)} \\
=& \ r(s, a)
+ \ E_{s_1 \sim p(.|s, a), a_1 \sim \pi(.|s_1)}[\gamma E_{p^{\pi}}[\sum_{t = 0}^{+\infty}\gamma^{t}r(s_t, a_t) \ | s_1, a_1]]  \\
=& \ E_{s^{'} \sim p(.|s, a), a^{'} \sim \pi(.|s^{'})}[r(s, a) + \gamma Q^\pi(s^{'}, a^{'})] \\
\end{align*}
<br>Thus :
> \begin{align*}
Q^*(s,a) =& \max_\pi Q^\pi(s, a) \\
=& \max_\pi \ E_{s^{'} \sim p(.|s, a), a^{'} \sim \pi(.|s^{'})}[r(s, a) + \gamma Q^\pi(s^{'}, a^{'})] \\
=& \ E_{s^{'} \sim p(.|s, a), a^{'} \sim \pi(.|s^{'})}[r(s, a) + \max_\pi \gamma Q^\pi(s^{'}, a^{'})] \\
=& \ E_{s^{'} \sim p(.|s, a)}[r(s, a) + \max_{a^{'}} \gamma Q^\pi(s^{'}, a^{'})]
\end{align*}
<br> According to what we just proved, we have that :
\begin{equation*}
\mathcal{L}(\theta)
= E_{s^{'} \sim p(.|s, a)} || r(s, a) + \gamma \max_{a^{'}} Q(s^{'}, a^{'}, \theta) - Q(s, a, \theta) ||^2 
= E_{\pi} || Q^*(s, a, \theta) - Q^\pi(s, a, \theta) ||^2 
\end{equation*}
where $Q^*(., ., \theta)$ acts as a proxy for the true $Q^*$. Our loss function is thus the L2 norm under the probability measure $\pi$ of $Q^*(., ., \theta) - Q^\pi(., ., \theta)$, which is a plausible objective.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            random_idx = np.random.choice(self.max_memory)
            self.memory[random_idx] = m

    def random_access(self):
        return self.memory[np.random.choice(len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [23]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        outputs = self.model.predict(np.expand_dims(s, axis=0))
        a = np.argmax(outputs)
        return a

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s, a_, r_, game_over = self.memory.random_access()
            input_states[i] = s_
            target_q[i] = self.model.predict(np.expand_dims(s_, axis = 0))[0]
            target_q[i, a_] = r_ + self.discount * np.max(self.model.predict(np.expand_dims(n_s, axis=0)))
            
            """
            if game_over_:
                ######## FILL IN
            else:
                ######## FILL IN
            """
            
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        in_shape = (5, 5, self.n_state)
        n_actions = 4
        
        model = Sequential()
        model.add(Dense(64, input_shape=in_shape, activation='relu', kernel_initializer = 'glorot_uniform'))
        model.add(Flatten())
        model.add(Dense(n_actions))
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.summary()
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [24]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 5, 5, 64)          192       
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 6404      
Total params: 6,596
Trainable params: 6,596
Non-trainable params: 0
_________________________________________________________________
Epoch 000/200 | Loss 0.0004 | Win/lose count 1.0/1.0 (0.0)
Epoch 001/200 | Loss 0.0069 | Win/lose count 3.5/6.0 (-2.5)
Epoch 002/200 | Loss 0.0086 | Win/lose count 3.5/4.0 (-0.5)
Epoch 003/200 | Loss 0.0022 | Win/lose count 6.5/2.0 (4.5)
Epoch 004/200 | Loss 0.0039 | Win/lose count 2.5/0 (2.5)
Epoch 005/200 | Loss 0.0016 | Win/lose count 3.5/2.0 (1.5)
Epoch 006/200 | Loss 0.0049 | Win/lose count 2.5/0

Epoch 125/200 | Loss 0.0014 | Win/lose count 22.0/5.0 (17.0)
Epoch 126/200 | Loss 0.0015 | Win/lose count 18.0/2.0 (16.0)
Epoch 127/200 | Loss 0.0009 | Win/lose count 23.5/4.0 (19.5)
Epoch 128/200 | Loss 0.0012 | Win/lose count 23.0/3.0 (20.0)
Epoch 129/200 | Loss 0.0008 | Win/lose count 19.0/3.0 (16.0)
Epoch 130/200 | Loss 0.0010 | Win/lose count 19.5/2.0 (17.5)
Epoch 131/200 | Loss 0.0011 | Win/lose count 19.5/2.0 (17.5)
Epoch 132/200 | Loss 0.0038 | Win/lose count 15.0/6.0 (9.0)
Epoch 133/200 | Loss 0.0009 | Win/lose count 24.0/6.0 (18.0)
Epoch 134/200 | Loss 0.0009 | Win/lose count 21.5/3.0 (18.5)
Epoch 135/200 | Loss 0.0007 | Win/lose count 4.0/2.0 (2.0)
Epoch 136/200 | Loss 0.0011 | Win/lose count 3.0/2.0 (1.0)
Epoch 137/200 | Loss 0.0004 | Win/lose count 21.0/2.0 (19.0)
Epoch 138/200 | Loss 0.0008 | Win/lose count 15.5/4.0 (11.5)
Epoch 139/200 | Loss 0.0006 | Win/lose count 24.0/2.0 (22.0)
Epoch 140/200 | Loss 0.0012 | Win/lose count 12.5/5.0 (7.5)
Epoch 141/200 | Loss 0.0052 | 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [25]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        in_shape = (5, 5, self.n_state)
        n_actions = 4
        
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=in_shape))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=in_shape))
        model.add(Flatten())
        model.add(Dense(n_actions))
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.summary()
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [26]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 5, 5, 32)          608       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 64)          18496     
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 6404      
Total params: 25,508
Trainable params: 25,508
Non-trainable params: 0
_________________________________________________________________
Epoch 000/200 | Loss 0.0019 | Win/lose count 3.0/5.0 (-2.0)
Epoch 001/200 | Loss 0.0007 | Win/lose count 2.5/1.0 (1.5)
Epoch 002/200 | Loss 0.0009 | Win/lose count 2.5/2.0 (0.5)
Epoch 003/200 | Loss 0.0034 | Win/lose count 2.0/1.0 (1.0)
Epoch 004/200 | Loss 0.0013 | Win

Epoch 123/200 | Loss 0.0012 | Win/lose count 11.5/4.0 (7.5)
Epoch 124/200 | Loss 0.0008 | Win/lose count 11.5/4.0 (7.5)
Epoch 125/200 | Loss 0.0026 | Win/lose count 12.5/3.0 (9.5)
Epoch 126/200 | Loss 0.0008 | Win/lose count 19.5/4.0 (15.5)
Epoch 127/200 | Loss 0.0012 | Win/lose count 7.0/1.0 (6.0)
Epoch 128/200 | Loss 0.0027 | Win/lose count 7.5/3.0 (4.5)
Epoch 129/200 | Loss 0.0010 | Win/lose count 20.5/3.0 (17.5)
Epoch 130/200 | Loss 0.0007 | Win/lose count 23.5/5.0 (18.5)
Epoch 131/200 | Loss 0.0027 | Win/lose count 19.0/1.0 (18.0)
Epoch 132/200 | Loss 0.0022 | Win/lose count 20.5/1.0 (19.5)
Epoch 133/200 | Loss 0.0010 | Win/lose count 14.5/2.0 (12.5)
Epoch 134/200 | Loss 0.0014 | Win/lose count 16.0/1.0 (15.0)
Epoch 135/200 | Loss 0.0009 | Win/lose count 6.5/3.0 (3.5)
Epoch 136/200 | Loss 0.0006 | Win/lose count 14.5/2.0 (12.5)
Epoch 137/200 | Loss 0.0016 | Win/lose count 2.5/3.0 (-0.5)
Epoch 138/200 | Loss 0.0010 | Win/lose count 16.0/0 (16.0)
Epoch 139/200 | Loss 0.0012 | Win/lo

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [27]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 5, 5, 32)          608       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 5, 5, 64)          18496     
_________________________________________________________________
flatten_4 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 6404      
Total params: 25,508
Trainable params: 25,508
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 5, 5, 64)          192       
_________________________________________________________________
flatte

In [28]:
HTML(display_videos('cnn_test10.mp4'))

In [29]:
HTML(display_videos('fc_test10.mp4'))

In [31]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 5, 5, 32)          608       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 5, 5, 64)          18496     
_________________________________________________________________
flatten_8 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 4)                 6404      
Total params: 25,508
Trainable params: 25,508
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 5, 5, 64)          192       
_________________________________________________________________
flatte

In [32]:
HTML(display_videos('cnn_test10.mp4'))

In [33]:
HTML(display_videos('fc_test10.mp4'))

> The two algorithms perform similary. With temperature equal to 0.3, their obtain the final scores of 13.34 for the convolutional network, and 13.5 for the fully connected network. With temperature equal to 0.5, the respective final scores become 12.89 and 12.51.
Both algorithms fail at efficiently exploring the map, particularly the convolutional network. A higher temperature leads to more exploration as well, as the density of cheese increases.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [44]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    epsilon_0 = agent.epsilon
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        eps = epsilon_0

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            agent.set_epsilon(eps)
            action = agent.act(state)
            eps *= 0.99

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [48]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        pass
    
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.visited = np.zeros((grid_size,grid_size)) # already visited cells

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0
        self.scale=16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.position[self.x, self.y] = 1
        
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.position[self.x, self.y] = 1
        
        self.visited[self.x, self.y] = -0.1 # malus for already visited cells
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        if train:
            reward += self.visited[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate(
            (self.board.reshape(self.grid_size, self.grid_size, 1),
            self.position.reshape(self.grid_size, self.grid_size, 1),
            self.visited.reshape(self.grid_size, self.grid_size, 1)),
            axis=2
        )
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.visited = np.zeros((self.grid_size, self.grid_size))
        self.visited[self.x, self.y] = -0.1

        state = np.concatenate(
            (self.board.reshape(self.grid_size, self.grid_size, 1),
            self.position.reshape(self.grid_size, self.grid_size, 1),
            self.visited.reshape(self.grid_size, self.grid_size, 1)),
            axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [49]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
# epsilon = 1 to allow for maximum exploration in the beginning, decaying it every epoch with multiplicative factor 0.99
# agent = DQN_CNN(size, lr=.1, epsilon = 1, memory_size=2000, batch_size = 32,n_state=3)
agent = DQN_FC(size, lr=.1, epsilon = 1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='fc_train_explore')
HTML(display_videos('fc_train_explore10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 5, 5, 64)          256       
_________________________________________________________________
flatten_13 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 4)                 6404      
Total params: 6,660
Trainable params: 6,660
Non-trainable params: 0
_________________________________________________________________
Epoch 000/200 | Loss 0.0126 | Win/lose count 7.0/24.500000000000096 (-17.500000000000096)
Epoch 001/200 | Loss 0.0055 | Win/lose count 8.0/17.69999999999999 (-9.699999999999989)
Epoch 002/200 | Loss 0.0084 | Win/lose count 10.0/23.000000000000075 (-13.000000000000075)
Epoch 003/200 | Loss 0.0142 | Win/lose count 7.5/26.000000000000128 (-18.500000000000128)
Epoch 004/200 | Loss 0.0041 | Win/lose count 

Epoch 084/200 | Loss 0.0060 | Win/lose count 23.0/24.900000000000087 (-1.9000000000000874)
Epoch 085/200 | Loss 0.0040 | Win/lose count 10.0/21.600000000000044 (-11.600000000000044)
Epoch 086/200 | Loss 0.0090 | Win/lose count 17.5/19.500000000000014 (-2.000000000000014)
Epoch 087/200 | Loss 0.0030 | Win/lose count 18.5/24.700000000000053 (-6.200000000000053)
Epoch 088/200 | Loss 0.0045 | Win/lose count 23.5/22.50000000000005 (0.9999999999999503)
Epoch 089/200 | Loss 0.0048 | Win/lose count 21.0/19.199999999999992 (1.8000000000000078)
Epoch 090/200 | Loss 0.0056 | Win/lose count 18.0/21.100000000000033 (-3.1000000000000334)
Epoch 091/200 | Loss 0.0037 | Win/lose count 19.5/20.800000000000036 (-1.3000000000000362)
Epoch 092/200 | Loss 0.0072 | Win/lose count 17.5/26.10000000000007 (-8.600000000000069)
Epoch 093/200 | Loss 0.0053 | Win/lose count 19.0/19.800000000000015 (-0.8000000000000149)
Epoch 094/200 | Loss 0.0045 | Win/lose count 14.5/25.700000000000077 (-11.200000000000077)
Epoch 

Epoch 176/200 | Loss 0.0077 | Win/lose count 17.0/22.70000000000005 (-5.700000000000049)
Epoch 177/200 | Loss 0.0044 | Win/lose count 13.0/21.40000000000005 (-8.400000000000048)
Epoch 178/200 | Loss 0.0064 | Win/lose count 24.5/23.700000000000045 (0.7999999999999545)
Epoch 179/200 | Loss 0.0053 | Win/lose count 22.5/15.699999999999976 (6.800000000000024)
Epoch 180/200 | Loss 0.0026 | Win/lose count 14.0/19.30000000000002 (-5.3000000000000185)
Epoch 181/200 | Loss 0.0082 | Win/lose count 26.0/11.599999999999977 (14.400000000000023)
Epoch 182/200 | Loss 0.0043 | Win/lose count 21.0/23.800000000000033 (-2.8000000000000327)
Epoch 183/200 | Loss 0.0047 | Win/lose count 23.5/18.199999999999992 (5.300000000000008)
Epoch 184/200 | Loss 0.0037 | Win/lose count 22.5/23.500000000000064 (-1.000000000000064)
Epoch 185/200 | Loss 0.0047 | Win/lose count 16.0/23.400000000000077 (-7.400000000000077)
Epoch 186/200 | Loss 0.0138 | Win/lose count 26.5/16.89999999999999 (9.600000000000009)
Epoch 187/200 |

In [50]:
# Evaluation
test(agent,env,epochs_test,prefix='fc_test_explore')
HTML(display_videos('fc_test_explore10.mp4'))

Win/lose count 13.0/21.60000000000004. Average score (-8.60000000000004)
Win/lose count 22.5/12.199999999999973. Average score (0.8499999999999934)
Win/lose count 23.0/8.299999999999986. Average score (5.466666666666668)
Win/lose count 15.0/17.299999999999986. Average score (3.525000000000004)
Win/lose count 19.5/16.599999999999977. Average score (3.400000000000008)
Win/lose count 28.0/12.79999999999997. Average score (5.366666666666679)
Win/lose count 30.5/11.89999999999998. Average score (7.257142857142871)
Win/lose count 22.0/18.4. Average score (6.800000000000012)
Win/lose count 20.5/13.299999999999972. Average score (6.844444444444458)
Win/lose count 20.0/14.699999999999976. Average score (6.690000000000015)
Win/lose count 16.5/16.799999999999986. Average score (6.05454545454547)
Win/lose count 24.0/13.499999999999979. Average score (6.425000000000016)
Win/lose count 19.5/14.599999999999968. Average score (6.307692307692325)
Win/lose count 14.0/19.700000000000006. Average score (5

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***